# Import Library

In [1]:
import requests
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

# Tool Function

In [2]:
DRUG_DIR = "drugs"
#Purpose: Defines the directory name where all drug information files will be stored locally.

In [3]:
def search_drug_info(drug_name: str, max_results: int = 5) -> List[str]:
    """
    Search for drug information from openFDA by drug name (brand or substance).
    
    Args:
        drug_name (str): The name of the drug to search (e.g. 'ibuprofen')
        max_results (int): Number of results to fetch (default: 5)

    Returns:
        List[str]: A list of drug brand names found
    """

    # Prepare search query (brand_name OR substance_name)
    url = f"https://api.fda.gov/drug/label.json?search=openfda.brand_name:{drug_name}+openfda.substance_name:{drug_name}&limit={max_results}"

    # Send request
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("results", [])

    # Create directory to store results
    path = os.path.join(DRUG_DIR, drug_name.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    file_path = os.path.join(path, "drug_info.json")

    # Collect drug information
    drug_info = {}
    brand_names = []

    for entry in results:
        openfda_data = entry.get("openfda", {})
        brand_name = openfda_data.get("brand_name", ["Unknown"])[0]
        substance = openfda_data.get("substance_name", ["Unknown"])[0]
        manufacturer = openfda_data.get("manufacturer_name", ["Unknown"])[0]
        route = openfda_data.get("route", ["Unknown"])[0]

        info = {
            "brand_name": brand_name,
            "substance_name": substance,
            "manufacturer": manufacturer,
            "route": route,
            "purpose": entry.get("purpose", ["Not specified"])[0],
            "usage": entry.get("indications_and_usage", ["Not specified"])[0],
            "warnings": entry.get("warnings", ["Not specified"])[0],
            "adverse_reactions": entry.get("adverse_reactions", ["Not specified"])[0],
            "boxed_warning": entry.get("boxed_warning", ["None"])[0]
        }

        drug_info[brand_name] = info
        brand_names.append(brand_name)

    # Save to file
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(drug_info, f, indent=2, ensure_ascii=False)

    print(f"\n Drug info saved at: {file_path}")
    print("\n Found the following:")
    for name in brand_names:
        print(f"- {name}")
    
    return brand_names

In [4]:
search_drug_info("insulin", max_results=1)


 Drug info saved at: drugs/insulin/drug_info.json

 Found the following:
- Pancreas Liquescence


['Pancreas Liquescence']

In [5]:
def extract_drug_info(brand_name: str) -> str:
    """
    Search for information about a specific drug by brand name across all drugs directories.
    Args:
        brand_name (str): The brand name of the drug to look for

    Returns:
        str: JSON-formatted string with drug info if found, or an error message if not found
    """

    for folder in os.listdir("drugs"):
        folder_path = os.path.join("drugs", folder)
        if os.path.isdir(folder_path):
            file_path = os.path.join(folder_path, "drug_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        if brand_name in data:
                            return json.dumps(data[brand_name], indent=2, ensure_ascii=False)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"No saved information found for drug brand: {brand_name}"

In [6]:
extract_drug_info("Pancreas Liquescence")

'{\n  "brand_name": "Pancreas Liquescence",\n  "substance_name": "CHIONANTHUS VIRGINICUS ROOT BARK",\n  "manufacturer": "Nutritional Specialties, Inc.",\n  "route": "ORAL",\n  "purpose": "PURPOSE: Provides potentized homeo-nutritional support of the pancreas.† †Claims based on traditional homeopathic practice, not accepted medical evidence. Not FDA evaluated.",\n  "usage": "INDICATIONS: Provides potentized homeo-nutritional support of the pancreas.† †Claims based on traditional homeopathic practice, not accepted medical evidence. Not FDA evaluated.",\n  "warnings": "WARNINGS: Professional Use Only If pregnant or breast-feeding, ask a health professional before use. In case of overdose, get medical help or contact a Poison Control Center right away. If condition worsens, seek medical attention. KEEP OUT OF REACH OF CHILDREN Do not use if tamper evident seal is broken or missing. Store in a cool place after opening",\n  "adverse_reactions": "Not specified",\n  "boxed_warning": "None"\n}'

# Tool Schema

In [7]:
tools = [
    {
        "name": "search_drug_info",
        "description": "Search for information about a drug from openFDA and store basic details into a file.",
        "input_schema": {
            "type": "object",
            "properties": {
                "drug_name": {
                    "type": "string",
                    "description": "The name of the drug to search for (e.g. ibuprofen, insulin) The main active ingredient; a generic, non-proprietary name."
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve (default is 5)",
                    "default": 5
                }
            },
            "required": ["drug_name"]
        }
    },
    {
        "name": "extract_drug_info",
        "description": "Retrieve information about a specific drug by its brand name from saved drug files. ",
        "input_schema": {
            "type": "object",
            "properties": {
                "brand_name": {
                    "type": "string",
                    "description": "The brand name of the drug to look for (e.g. ADMELOG). The brand name of the product, created and trademarked by the company."
                }
            },
            "required": ["brand_name"]
        }
    }
]

# Tool Mapping

In [8]:
mapping_tool_function = {
    "search_drug_info": search_drug_info,
    "extract_drug_info": extract_drug_info
}
def execute_tool(tool_name, tool_args):
    """
    Execute a registered tool by name with provided arguments.
    
    Args:
        tool_name (str): The name of the tool to call
        tool_args (dict): Dictionary of arguments for the tool

    Returns:
        str: Stringified and formatted result for display
    """

    try:
        result = mapping_tool_function[tool_name](**tool_args)
    except Exception as e:
        return f"Error while executing tool: {str(e)}"

    # Format result for display
    if result is None:
        return "Operation completed, but no data was returned."

    elif isinstance(result, list):
        return "\n".join(f"- {item}" for item in result)

    elif isinstance(result, dict):
        return json.dumps(result, indent=2, ensure_ascii=False)

    else:
        return str(result)

# Set up AI Client

In [9]:
load_dotenv()
client = anthropic.Anthropic()

# Query Processing 

In [10]:
def process_query(query):
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages)
    
    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:
            if content.type == 'text':
                
                print(content.text)
                assistant_content.append(content)
                
                if len(response.content) == 1:
                    process_query = False
            
            elif content.type == 'tool_use':
                
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})
                
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                messages.append({"role": "user", 
                                  "content": [
                                      {
                                          "type": "tool_result",
                                          "tool_use_id": tool_id,
                                          "content": result
                                      }
                                  ]
                                })
                response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages) 
                
                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    process_query = False

# Chat Loop

In [11]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [ ]:
chat_loop()